In [ ]:
import datasets
import os
import numpy as np
import sys
sys.path.append("../retrievers")
import texar as tx

from tqdm import tqdm
from totto_retriever import TottoRetriever


os.environ["HF_HOME"] = "/projects/ogma2/users/andrewsi/cache/huggingface/"
root = "/projects/ogma2/users/andrewsi/control-data2text"

In [ ]:
val_json = f"{root}/transformers/examples/seq2seq/test_data/totto/val_with_headers_only.json"
preds_file = ""

with open(preds_file, "r") as f:
    normal_preds = [line.strip() for line in f]
mask_token = "<UNK>"
masked_preds = []
masked_protos = []
val_data = datasets.load_dataset("json", data_files=val_json)["train"]
for i in tqdm(range(len(normal_preds))):
    proto, source = val_data[i]["source"].split("[SEP]")
    proto = proto.strip()
    source = source.strip()
    masked_preds.append(TottoRetriever.mask_target(source, normal_preds[i], mask_token=mask_token).strip())
    masked_protos.append(TottoRetriever.mask_target(source, proto, mask_token=mask_token).strip())

In [ ]:
bleu_refs = [[ref] for ref in masked_protos]
bleu_preds = list(map(str.split, masked_preds))
m_bleu = tx.evals.corpus_bleu_moses(bleu_refs, bleu_preds, lowercase=True, return_all=False)
print(f"m_bleu: {m_bleu}")